In [ ]:
## Quiz Generator from ai-assistant-teddy 
## url: https://teddylee777.github.io/openai/openai-assistant-tutorial/#google_vignette

퀴즈 생성기 전체코드
1) 환경설정Permalink
.env 파일로부터 API KEY를 불러옵니다

In [ ]:
# API KEY 정보를 불러옵니다
from dotenv import load_dotenv
from openai import OpenAI
from functions import *
load_dotenv()

In [ ]:
import os

# os.environ["OPENAI_API_KEY"] = "API KEY를 입력해 주세요"
# OPENAI_API_KEY 를 설정합니다.
api_key = os.environ.get("OPENAI_API_KEY")
print(api_key)

In [5]:
# OpenAI API를 사용하기 위한 클라이언트 객체를 생성합니다.
client = OpenAI(api_key=api_key)

2) 파일 업로드Permalink
출제될 문제가 기반할 파일을 업로드합니다.

신규 파일 업로드
기존 업로드된 File ID 사용
신규 파일 업로드를 위한 코드입니다. 이미 업로드한 경우 건너 뛰어도 됩니다.

In [ ]:
# 파일 업로드를 위한 함수를 정의합니다.
def upload_files(files):
    uploaded_files = []
    for filepath in files:
        file = client.files.create(
            file=open(
                # 업로드할 파일의 경로를 지정합니다.
                filepath,  # 파일경로. (예시) data/sample.pdf
                "rb",
            ),
            purpose="assistants",
        )
        uploaded_files.append(file)
        print(f"[업로드한 파일 ID]\n{file.id}")
    return uploaded_files


# 필요에 따라서는 파일을 업로드 합니다.
# 파일 업로드시 아래 주석을 해제하고 업로드할 파일의 경로를 지정합니다.
# 업로드할 파일들의 경로를 지정합니다.
# files_to_upload = [
#     "data/language_models_are_unsupervised_multitask_learners.pdf",
#     "data/SPRI_AI_Brief_2023년12월호.pdf",
# ]

# 파일을 업로드합니다.
# file_ids = upload_files(files_to_upload)

In [ ]:
#  이미 업로드한 모든 파일 ID 와 파일명을 출력하여 file.id 목록을 file_ids 에 저장
file_ids = []
for file in client.files.list():
    file_ids.append(file.id)
    print(f"[파일 ID] {file.id} [파일명] {file.filename}")
print(file_ids)

In [9]:
# Assistant 가 참고할 파일 ID를 지정합니다.
file_ids = [file_ids[0], file_ids[1], file_ids[2]]

3) Function Schema 정의Permalink
다음으로는 Assistant 를 생성합니다. 생성시 tools에 function을 추가합니다.

In [7]:
# 스키마를 정의합니다.
function_schema = {
    "name": "generate_quiz",
    "description": "Generate a quiz to the student, and returns the student's response. A single quiz has multiple questions.",
    "parameters": {
        "type": "object",
        "properties": {
            "title": {"type": "string"},
            "questions": {
                "type": "array",
                "description": "An array of questions, each with a title and multiple choice options.",
                "items": {
                    "type": "object",
                    "properties": {
                        "question_text": {"type": "string"},
                        "choices": {"type": "array", "items": {"type": "string"}},
                    },
                    "required": ["question_text", "choices"],
                },
            },
        },
        "required": ["title", "questions"],
    },
}

In [ ]:
# 퀴즈를 출제하는 역할을 하는 챗봇을 생성합니다.

assistant = client.beta.assistants.create(
    name="Quiz Generator",
    instructions="You are an expert in generating multiple choice quizzes. Create quizzes based on uploaded files.",
    model="gpt-4-turbo-preview",
    tools=[
        {"type": "code_interpreter"},
        {"type": "function", "function": function_schema},
    ],
)

ASSISTANT_ID = assistant.id
# 생성된 챗봇의 정보를 JSON 형태로 출력합니다.
show_json(assistant)

In [ ]:
# Add tools into ai_assistant use assistant.upate()
ASSISTANT_ID = assistant
print(ASSISTANT_ID)
assistant = client.beta.assistants.update(
    ASSISTANT_ID,
    tools=[
        {"type": "code_interpreter"},
        ],
    tool_resources = [
        {"type": "code_interpreter"},
        {"file_ids" : file_ids},]
)